# Releasing an assignment

**These instructions are currently incomplete**.

In [ ]:
import os
from nbgrader.utils import run

In [ ]:
os.chdir('example')

In [ ]:
run('ls -R')

In [ ]:
# remove an existing database
if os.path.exists("gradebook.db"):
    os.remove("gradebook.db")

# create a connection to the db using the nbgrader API
from nbgrader.api import Gradebook
gb = Gradebook("sqlite:///gradebook.db")

In [ ]:
# add the assignment to the database
gb.add_assignment("Problem Set 1")

In [ ]:
run('nbgrader assign --assignment="Problem Set 1" --IncludeHeaderFooter.header=source/header.ipynb')

After doing this, we should be able to see the grade cells in the database:

In [ ]:
# print out how many 
notebook = gb.find_notebook("Problem 1", "Problem Set 1")
print("There are {} grade cells in Problem Set 1, Problem 1".format(len(notebook.grade_cells)))